# PSPNetによる訓練

## 準備

### ライブラリのインポート

In [ ]:
import os
import math
import tqdm
import torch
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader

from pspnet import PSPNet
from dataset import VOCDataset
from util import DiceCrossEntropyLoss

### 既定変数
基本的に変更の必要なし

In [ ]:
# VOC2012で用いるラベル
CLASSES = ['backgrounds','aeroplane','bicycle','bird','boat','bottle',
            'bus','car' ,'cat','chair','cow', 
            'diningtable','dog','horse','motorbike','person', 
            'potted plant', 'sheep', 'sofa', 'train', 'monitor','unlabeld'
            ]

# カラーパレットの作成
COLOR_PALETTE = np.array(Image.open("./VOCdevkit/VOC2012_sample/SegmentationClass/2007_000170.png").getpalette()).reshape(-1,3)
COLOR_PALETTE = COLOR_PALETTE.tolist()[:len(CLASSES)]

# データディレクトリ
img_dir = 'VOCdevkit/VOC2012_sample/JPEGImages'
gt_dir = 'VOCdevkit/VOC2012_sample/SegmentationClass'

initial_epoch = 0

device = 'cuda' if torch.cuda.is_available() else "cpu" # デバイスの設定
print("Using {} device".format(device))

model = PSPNet(n_classes=22) # モデル
model = model.to(device)

### 実行周りの設定

In [ ]:
# データのリストファイル
train_list_path = 'VOCdevkit/VOC2012_sample/listfile/train_list_300.txt'
val_list_path = 'VOCdevkit/VOC2012_sample/listfile/val_list_100.txt'

# 保存先
save_dir = './pspnet_sample' # 訓練ログ保存ディレクトリ
save_name_prefix = 'pspnet_sample' # 訓練ログ保存名
csv_path = os.path.join(save_dir, f'{save_name_prefix}.csv')

# データローダーの引数（大きいほど高速に動作するが、GPUの性能により限界がある）
batch_size=16 # バッチサイズ
num_workers=8 # ワーカープロセス数（データ読み込みの並列数）

### 学習周りの設定

In [ ]:
# 学習関連
epochs = 100 # エポック
lr = 0.01 # 学習率

# 重み
chkp_path = None # 重みの初期値
continuation = False # 訓練を続きから再開する場合True

# データアーギュメント
data_augmentation = False

# 損失関数関連
weight = None
dice_weight = 0.5
aux_weight = 0.4

### データの読み込み、損失関数・最適化手法の定義、重みの読み込み

In [ ]:
# リストファイルの読み込み
with open(train_list_path, 'r') as f:
    train_list = f.read().splitlines()
with open(val_list_path, 'r') as g:
    val_list = g.read().splitlines()

# データセットの作成
train_ds = VOCDataset(img_list=train_list, img_dir=img_dir, gt_dir=gt_dir, data_augmentation=data_augmentation)
val_ds = VOCDataset(img_list=val_list, img_dir=img_dir, gt_dir=gt_dir)
print(f"len(train_data): {train_ds.__len__()}")
print(f"len(test_data): {val_ds.__len__()}")

# データローダーの作成
train_dl = DataLoader(train_ds, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# 保存ディレクトリの作成
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 最適化手法
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

# 重みの読み込み。
if chkp_path is not None:
    checkpoint = torch.load(chkp_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    if continuation == True:
        initial_epoch = checkpoint["epoch"] + 1

if weight is not None:
    weight = weight.to(device)
criterion =  DiceCrossEntropyLoss(weight=weight, dice_weight=dice_weight) # 損失関数

### 関数

In [ ]:
# 訓練
def train(dataloader, model, optimizer, criterion, aux_weight, device):
    model.train()
    size = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    step_total = math.ceil(size/batch_size)
    total_loss = 0.0

    with tqdm.tqdm(enumerate(dataloader), total=step_total) as pbar:
        for batch, item in pbar:
            inp, gt = item[0].to(device), item[1].to(device)

            # 推論
            pred, pred_aux = model(inp)

            # 損失誤差を計算
            loss = criterion(pred, gt) + aux_weight * criterion(pred_aux, gt)
            total_loss += loss.item()

            # バックプロパゲーション
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # プログレスバーに損失を表示
            pbar.set_postfix(loss=loss.item())

        avg_loss = total_loss / step_total
        print(f"Train Loss: {avg_loss:.4f}")
    return avg_loss

# テスト
def test(dataloader, model, criterion, aux_weight, device):
    model.eval()
    size = len(dataloader.dataset)
    batch_size = dataloader.batch_size
    step_total = math.ceil(size/batch_size)
    total_loss = 0.0

    with torch.no_grad():
        for item in dataloader:
            inp, gt = item[0].to(device), item[1].to(device)
            pred, pred_aux = model(inp)
            loss = criterion(pred, gt) + aux_weight * criterion(pred_aux, gt)
            total_loss += loss.item()

        avg_loss = total_loss / step_total
        print(f"Val Loss: {avg_loss:.4f}")
    return avg_loss

# 推論結果の可視化
def visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight, device='cpu'):
    img_path = os.path.join(img_dir, img_id) + ".jpg"
    gt_path = os.path.join(gt_dir, img_id) + ".png"
    
    model.to(device)
    model.eval()

    with torch.no_grad():
        # 入力画像の前処理 (PIL画像 -> Tensor)
        inp_tensor = transforms.functional.to_tensor(Image.open(img_path))
        inp_resized = transforms.Resize(size=(256, 256), interpolation=transforms.InterpolationMode.NEAREST)(inp_tensor)
        inp_normalized = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(inp_resized)
        inp = inp_normalized.unsqueeze(0)

        # 正解画像の前処理 (PIL画像 -> NumPy -> Tensor)
        gt_np = np.asarray(Image.open(gt_path))
        gt_np = np.where(gt_np == 255, 0, gt_np)  # 範囲外の255を0に置換（または別のインデックスに）

        gt_tensor = torch.tensor(gt_np, dtype=torch.long)
        gt_tensor = gt_tensor.unsqueeze(0)
        gt_resized = transforms.Resize(size=(256, 256), interpolation=transforms.InterpolationMode.NEAREST)(gt_tensor)

        # 予測
        pred, pred_aux = model(inp)
        loss = criterion(pred, gt_resized) + aux_weight * criterion(pred_aux, gt_resized) # loss計算
        print(f'Loss: {loss.item()}')

        inp_np = (inp_resized.numpy().transpose(1, 2, 0) * 255).astype(np.uint8)  # 表示用スケール
        pred_np = torch.argmax(pred, dim=1).cpu().detach().numpy()[0]
        pred_np = np.clip(pred_np, 0, len(COLOR_PALETTE) - 1).astype(np.int32)
        gt_resized = gt_resized.squeeze(0).detach().numpy().astype(np.int32)

        # カラーパレットの適用
        img_gt = np.array([[COLOR_PALETTE[gt_resized[i, j]] for j in range(256)] for i in range(256)], dtype=np.uint8)
        img_pred = np.array([[COLOR_PALETTE[pred_np[i, j]] for j in range(256)] for i in range(256)], dtype=np.uint8)

    # プロット
    fig = plt.figure(figsize=(16, 10))
    for i, im in enumerate([inp_np, img_gt, img_pred]):
        ax = fig.add_subplot(1, 3, i+1)
        ax.imshow(im)
        ax.set_title(["Input", "Ground Truth", "Prediction"][i])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

## 訓練

In [ ]:
for epoch in range(initial_epoch, epochs):
    print(f"Epoch {epoch}\n-------------------------------")
    train_loss = train(train_dl, model, optimizer, criterion, aux_weight, device)
    val_loss = test(val_dl, model, criterion, aux_weight, device)

    # モデルの重みを保存
    torch.save(
    {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        },
    os.path.join(save_dir, f'{save_name_prefix}_ep{epoch}.pth'),
    )

    # 訓練ログ記入
    if not os.path.exists(csv_path):
        with open(csv_path, 'w') as f:
            f.write('epoch,train_loss,val_loss\n')
    with open(csv_path, 'a') as f:
        f.write(f'{epoch},{train_loss},{val_loss}\n')

## 可視化

### 損失のプロット

In [ ]:
log_df = pd.read_csv(csv_path)
plt.plot(log_df['epoch'], log_df['train_loss'], label='trian_data')
plt.plot(log_df['epoch'], log_df['val_loss'], label='test_data')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
# plt.ylim(0,2.0)

### 推論結果

<b>train data</b>

In [ ]:
img_id = '2007_000032' # 好きな画像IDを入れてOK
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

In [ ]:
img_id = '2007_000648'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

In [ ]:
img_id = '2007_001225'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

In [ ]:
img_id = '2007_002488'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

<b>test data</b>

In [ ]:
img_id = '2007_000033'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

In [ ]:
img_id = '2007_001763'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)

In [ ]:
img_id = '2007_003367'
visualize(model, img_id, img_dir, gt_dir, criterion, aux_weight)